In [1]:
#librerias
library(dplyr)
library(lubridate)


Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Attaching package: ‘lubridate’

The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union



# Libreta ejecutada en R

Los datos trabjados so obtenidos de forma libre de la página de la <a href='http://www.aire.cdmx.gob.mx/default.php?opc=%27aKBhnmI=%27&opcion=Zw=='> Ciudad de México</a>, se obtienen diferentes indices y valores desde el año 2000 hasta octubre de 2020.

Se descargaron los datos en archivos CSV, cada archivo representa un año de medición. Para este analisis se descagaron datos meteorológicos para obtener información de la temperatura y datos de contaminantes para obtener valores de **NOx, SO2, CO**

In [19]:
estaciones <- read.csv('data/cat_estacion.csv')
parametros <- read.csv('data/cat_parametros.csv')
unidades <- read.csv('data/cat_unidades.csv')

In [3]:
# Lectura de archivo 
dm00 <- read.csv('data/meteorología_2000.csv',skip = 10, header = T)
dc00 <- read.csv('data/contaminantes_2000.csv',skip = 10, header = T)
#class(dm00)
#class(dc00)
head(dc00)
head(dm00)

date,cve_station,cve_parameter,value,unit
01/01/2000 01:00,LAG,CO,2.3,15
01/01/2000 01:00,TAC,CO,1.0,15
01/01/2000 01:00,FAC,CO,NA,15
01/01/2000 01:00,TLA,CO,3.7,15
01/01/2000 01:00,XAL,CO,6.2,15
01/01/2000 01:00,MER,CO,2.8,15


date,cve_station,cve_parameter,value,unit
01/01/2000 01:00,TAC,RH,NA,6
01/01/2000 01:00,FAC,RH,NA,6
01/01/2000 01:00,SAG,RH,NA,6
01/01/2000 01:00,TLA,RH,38,6
01/01/2000 01:00,XAL,RH,51,6
01/01/2000 01:00,MER,RH,47,6


In [4]:
# Mascara de archivos a trabajar
nox <- dc00['cve_parameter'] == 'NOX'
co <- dc00['cve_parameter'] == 'CO'
so2 <- dc00['cve_parameter'] == 'SO2'
pm2 <- dc00['cve_parameter'] == 'PM2.5'
tmp <- dm00['cve_parameter'] == 'TMP'

In [5]:
# Cambiamos de factor a fecha
#df['date'] <- df %>% select(date) %>% mutate(date = dmy_hms(as.character(date)))
dm00['date'] <- dm00 %>% select(date) %>% mutate(date = dmy_hms(as.character(date)))
dc00['date'] <- dc00 %>% select(date) %>% mutate(date = dmy_hms(as.character(date)))
head(dm00)

date,cve_station,cve_parameter,value,unit
2020-01-01 00:01:00,TAC,RH,NA,6
2020-01-01 00:01:00,FAC,RH,NA,6
2020-01-01 00:01:00,SAG,RH,NA,6
2020-01-01 00:01:00,TLA,RH,38,6
2020-01-01 00:01:00,XAL,RH,51,6
2020-01-01 00:01:00,MER,RH,47,6


In [14]:
# Agrupamos los datos por día para obtener promedios
dftmp <- dm00 %>% mutate(month = month(date), day = day(date)) %>% group_by(month, day)
dftmp <- dftmp[tmp,]
head(dftmp)

Warning message:
“The `i` argument of ``[`()` can't be a matrix as of tibble 3.0.0.
Convert to a vector.
This warning is displayed once every 8 hours.
Call `lifecycle::last_warnings()` to see where this warning was generated.”

date,cve_station,cve_parameter,value,unit,month,day
2020-01-01 00:01:00,TAC,TMP,-1.0,5,1,1
2020-01-01 00:01:00,FAC,TMP,-1.0,5,1,1
2020-01-01 00:01:00,SAG,TMP,10.1,5,1,1
2020-01-01 00:01:00,TLA,TMP,9.6,5,1,1
2020-01-01 00:01:00,XAL,TMP,10.8,5,1,1
2020-01-01 00:01:00,MER,TMP,11.9,5,1,1


In [7]:
# Eliminamos los NA
dm00[is.na(dm00)] <- -1

In [11]:
nNA <- dm00['value'] != -1
dm00[nNA,]

,date,cve_station,cve_parameter,value,unit
4,2020-01-01 00:01:00,TLA,RH,38,6
5,2020-01-01 00:01:00,XAL,RH,51,6
6,2020-01-01 00:01:00,MER,RH,47,6
8,2020-01-01 00:01:00,CES,RH,36,6
9,2020-01-01 00:01:00,PLA,RH,32,6
18,2020-01-01 00:02:00,TLA,RH,39,6
19,2020-01-01 00:02:00,XAL,RH,55,6
20,2020-01-01 00:02:00,MER,RH,47,6
22,2020-01-01 00:02:00,CES,RH,37,6
23,2020-01-01 00:02:00,PLA,RH,34,6


In [12]:
# Obtenemos promedios diarios


In [13]:
# Obtenemos promedios mensuales


In [15]:
# Creamos una matriz que almacena el promedio mensual de temperatura de las estaciones
# [date, [estacion]]

In [52]:
lista = estaciones[,1]
lista

[1] cve_estac ACO       AJU       AJM       ARA       ATI       AZC      
 [8] BJU       CAM       CCA       CES       CFE       CHO       COR      
[15] COY       CUA       CUI       CUT       DIC       EAJ       EDL      
[22] FAC       FAN       GAM       HAN       HGM       IBM       IMP      
[29] INN       IZT       LAA       LAG       LLA       LOM       LPR      
[36] LVI       MCM       MER       MGH       MIN       MON       MPA      
[43] NET       NEZ       PED       PER       PLA       POT       SAG      
[50] SFE       SHA       SJA       SNT       SUR       TAC       TAH      
[57] TAX       TEC       TLA       TLI       TPN       UAX       UIZ      
[64] UNM       VAL       VIF       XAL       XCH       FAR       SAC      
70 Levels: ACO AJM AJU ARA ATI AZC BJU CAM CCA CES CFE CHO COR COY CUA ... XCH

In [44]:
for(station in lista)
{
    tmp <- dftmp['cve_station']==station
    
}

In [45]:
tmp

cve_station
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
FALSE
